### some useful functions to figure out which settings to use

In [1]:
from nbodykit.cosmology import Planck15
import numpy as np

In [2]:
def a2z(a):
    return 1./a-1

def z2a(z):
    return 1./(1.+z)

In [3]:
def get_physical_width(cosmology,z_source,fov):
    """returns physical extent (comoving angular diameter distance) in Mpc/h 
    of a field of view (fov,in degrees) 
    at source redhsift z_source for a specific cosmology"""
    chi_source = cosmology.angular_diameter_distance(z_source)
    fov_rad    = fov/180.*np.pi
    width      = fov_rad*chi_source
    
    return width*(1+z_source)

In [4]:
def get_volume(z_source,FOV, cosmo):
    """
    returns the comoving volume of the lightcone in [Mpc/h]^3
    z_source: highest source redshift
    FOV: field of view in degrees (2D, in degrees)
    """
    h = cosmo.comoving_distance(z_source)
    ls= [deg/180.*np.pi*h for deg in FOV]
    V = np.prod(ls)*h/3.
    return V

In [5]:
def get_fov(cosmo,BoxSize,z_source):
    """
    get the field of view (in degrees) for given boxsize and source redshift
    """
    chi_source = cosmo.angular_diameter_distance(z_source)*(1+z_source)
    fov        = BoxSize/chi_source/np.pi*180.
    return fov

In [6]:
def get_num_boxes(cosmo, boxsize, z_source):
    """
    number of box repetitions up source redshift
    """
    h   = cosmo.comoving_distance(z_source)
    num = h/boxsize
    return num

In [7]:
cosmo_fid  = Planck15
cosmo_min  = Planck15.match(Omega0_m=0.05)
cosmo_max  = Planck15.match(Omega0_m=0.95)
boxsize    = 156
z_source   = 1.

print(get_fov(cosmo_fid,boxsize,z_source))
print(get_fov(cosmo_min,boxsize,z_source))
print(get_fov(cosmo_max,boxsize,z_source))

print(get_num_boxes(cosmo_fid,boxsize,z_source))
print(get_num_boxes(cosmo_min,boxsize,z_source))
print(get_num_boxes(cosmo_max,boxsize,z_source))

3.885479663282639
3.171171050261748
5.018371727603926
14.746127757280815
18.067703886803148
11.417205146484276


In [11]:
cosmo_fid  = Planck15
cosmo_min  = Planck15.match(Omega0_m=0.05)
cosmo_max  = Planck15.match(Omega0_m=0.95)
boxsize    = 256
z_source   = 1.

print(get_fov(cosmo_fid,boxsize,z_source))
print(get_fov(cosmo_min,boxsize,z_source))
print(get_fov(cosmo_max,boxsize,z_source))

print(get_num_boxes(cosmo_fid,boxsize,z_source))
print(get_num_boxes(cosmo_min,boxsize,z_source))
print(get_num_boxes(cosmo_max,boxsize,z_source))

6.3761717551304855
5.20397300555774
8.235276681196186
8.985921602092997
11.010007056020669
6.957359386138855


In [9]:
Planck15.Omega_m(0.)

0.3089037414433296

In [10]:
Planck15.sigma8

0.8158116144901115